In [1]:
import pandas as pd
import datetime
from configparser import ConfigParser
from pysimplicate import Simplicate

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [2]:
def fraction_of_the_year_past(start_day=None):
    if start_day:
        start_date_time = datetime.datetime.combine(start_day, datetime.datetime.min.time())
    else:
        y = datetime.datetime.today().year
        start_date_time = datetime.datetime(y, 1, 1)
    days_in_the_year = (datetime.datetime.today() - start_date_time).days + 1
    return days_in_the_year / 365


In [3]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
employees = sim.to_pandas(sim.timetable()).query(f"(end_date != end_date) or (end_date>'{today}')").employee_name.unique() # (end_date != end_date) is to check for NaN
full_balance_list = sim.to_pandas(sim.leavebalance())
current_balance_list = full_balance_list[ full_balance_list.employee_name.isin(employees)].query('leavetype_affects_balance==True')
current_balance_list = current_balance_list[current_balance_list['employee_name'] != 'Filipe José Mariano dos Santos']
current_balance_list

,employee_id,employee_name,balance,first_change,last_change,year,leavetype_id,leavetype_label,leavetype_affects_balance
1,employee:784ac55610fbd36e6d44e34a3f0f8c3d,Sho Stegmeijer,120.00,NaN,NaN,2020,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
2,employee:784ac55610fbd36e6d44e34a3f0f8c3d,Sho Stegmeijer,200.00,NaN,NaN,2021,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
4,employee:b5836dbd835040e56d44e34a3f0f8c3d,Rogier Voogt,88.00,NaN,NaN,2020,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
5,employee:b5836dbd835040e56d44e34a3f0f8c3d,Rogier Voogt,160.00,NaN,NaN,2021,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
7,employee:14b7ec842b5e2ec86d44e34a3f0f8c3d,Robin Veer,216.00,2021-04-21 09:30:00,2021-04-23 09:30:00,2021,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
...,...,...,...,...,...,...,...,...,...
229,employee:9d28f409c6c647fc6d44e34a3f0f8c3d,Joost Cornelissen,180.00,NaN,NaN,2021,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
235,employee:216a06d028f48da96d44e34a3f0f8c3d,Wouter Gerrits,0.02,2018-07-13 09:00:00,2018-12-31 08:30:00,2018,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
236,employee:216a06d028f48da96d44e34a3f0f8c3d,Wouter Gerrits,49.15,2019-02-05 09:30:00,2019-12-31 09:00:00,2019,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True
237,employee:216a06d028f48da96d44e34a3f0f8c3d,Wouter Gerrits,19.80,2020-03-17 09:00:00,2020-08-14 09:00:00,2020,leavetype:a2c2abddb4beab48674d3889da2d293d,Verlof / Leave,True


In [10]:
year = datetime.datetime.today().year
frac = fraction_of_the_year_past()
last_year = current_balance_list.query(f'year<{year}').groupby(['employee_name']).sum('balance')['balance'] / 8

    # This year
fulll_thisyear_list = sim.to_pandas(sim.leave({'year':year}))
current_thisyear_list = fulll_thisyear_list[fulll_thisyear_list.employee_name.isin(employees)].query(
    'leavetype_affects_balance==True'
)
this_year_balance = current_thisyear_list.groupby(['employee_name']).sum('hours')['hours'] / 8
this_year_new = current_thisyear_list.groupby(['employee_name']).max('hours')['hours'] / 8
overview = pd.concat([last_year, this_year_new, this_year_balance], axis=1).fillna(0)
overview.columns = ['last_year', 'this_year_new', 'this_year_balance']
overview['available'] = overview.apply(lambda x: x['last_year'] + x['this_year_balance'], axis=1)
overview['pool'] = overview.apply(lambda x: x['last_year'] + x['this_year_new'] * frac, axis=1)
overview.reset_index(level=0, inplace=True)
overview

,employee_name,last_year,this_year_new,this_year_balance,available,pool
0,Bas Colenbrander,2.00000,25.00,24.3125,26.31250,7.616438
1,Caspar Geerlings,10.00000,25.00,23.0000,33.00000,15.616438
2,Chris Neven,1.50000,25.00,23.0000,24.50000,7.116438
3,Eva Korteweg,-1.50000,22.50,21.5000,20.00000,3.554795
4,Fadhlur Zahri,2.75000,25.00,25.0000,27.75000,8.366438
5,Geert-Jan van Mastrigt,13.90000,15.00,7.0000,20.90000,17.269863
6,George Tsimenis,12.00000,25.00,24.0000,36.00000,17.616438
7,Gerben van Dijk,3.75000,25.00,17.0000,20.75000,9.366438
8,Gert Braun,14.50000,24.00,15.1875,29.68750,19.891781
9,Gijs Kattenberg,3.75000,25.00,23.5000,27.25000,9.366438


In [103]:
year = datetime.datetime.today().year
frac = fraction_of_the_year_past()
year_start = current_balance_list.query(f'year<{year}').groupby(['employee_name']).sum('balance')['balance']/8
this_year = current_balance_list.query(f'(year=={year}) and (balance>0)').groupby(['employee_name']).sum('balance')['balance']/8
used = current_balance_list.query(f'(year=={year}) and (balance<0)').groupby(['employee_name']).sum('balance')['balance']/-8
overview = pd.concat([year_start, this_year, used], axis=1).fillna(0)
overview.columns=['year_start','this_year','used']
overview['available'] = overview.apply( lambda x: x['year_start'] + x['this_year'] + x['used'], axis=1 )
overview['should_be_used'] = overview.apply( lambda x: x['this_year'] * frac, axis=1  )
overview['pool'] = overview.apply( lambda x: x['year_start'] + x['should_be_used'] - x['used'], axis=1)
overview.reset_index(level=0, inplace=True)
overview

,employee_name,year_start,this_year,used,available,should_be_used,pool
0,Bas Colenbrander,2.00000,24.50,0.1875,26.68750,5.101370,6.913870
1,Caspar Geerlings,10.00000,23.00,0.0000,33.00000,4.789041,14.789041
2,Chris Neven,1.50000,25.00,0.0000,26.50000,5.205479,6.705479
3,Eva Korteweg,-1.50000,21.50,0.0000,20.00000,4.476712,2.976712
4,Fadhlur Zahri,2.75000,25.00,0.0000,27.75000,5.205479,7.955479
5,Geert-Jan van Mastrigt,13.90000,7.00,0.0000,20.90000,1.457534,15.357534
6,George Tsimenis,12.00000,24.00,0.0000,36.00000,4.997260,16.997260
7,Gerben van Dijk,3.75000,23.00,0.0000,26.75000,4.789041,8.539041
8,Gert Braun,14.50000,11.50,0.3125,26.31250,2.394521,16.582021
9,Gijs Kattenberg,3.75000,23.50,0.0000,27.25000,4.893151,8.643151


In [102]:
overview['pool'].sum()

435.97462328767125